<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpusdev/blob/main/bibliography_dataset_download_v03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting bibliography for the Y1600, Y1700, Y1800 periods from library catalogues

## 1.1 downloading the json dump of vd17
This will last about 2 minutes, the command downloads 307 json files
(such files should be generated on stage 2, see 2.1, etc. -- but there are differences between the dupm and the downloaded version)

In [1]:
!git clone https://git.hab.de/beyer/vd17-dump/

Cloning into 'vd17-dump'...
remote: Enumerating objects: 4739, done.
remote: Counting objects: 100% (2862/2862), done.
remote: Compressing objects: 100% (660/660), done.
remote: Total 4739 (delta 2202), reused 2862 (delta 2202), pack-reused 1877
Receiving objects: 100% (4739/4739), 646.03 MiB | 18.95 MiB/s, done.
Resolving deltas: 100% (4033/4033), done.
Updating files: 100% (309/309), done.


now we try to filter the json objects, where

"langOrig":"eng",

e.g., /content/vd17-dump/json/vd17-290.json

	Line  6361:   "langOrig":"eng",
	Line 39193:   "langOrig":"eng",

and write this to a separate file

(ideally we try to create a separate language file for each langOrig value)

https://stackoverflow.com/questions/27189892/how-to-filter-json-array-in-python


### class to read the directory with json files and extract the needed language


In [13]:
import json
import re, os, sys

class clJsonDirFindFilter(object):
    '''
    @author Bogdan Babych, IÜD, Heidelberg University, 2023
    @email bogdan [dot] babych [at] iued [dot] uni-heidelberg [dot] de
    '''
    def __init__(self, SDirName, output_file = 'vdExtracted-all.json', findKey = 'langOrig', filtVal = 'eng'):
        self.FOut = open(output_file, 'w')

        self.output_data = []
        self.listVals = []

        self.mainDirWalk(SDirName, findKey)
        output_list = self.filtByLang(self.output_data, findKey, filtVal)

        self.dumpOutput(output_list, self.FOut)
        print(self.listVals)



    def mainDirWalk(self, path, findKey):

        for root,d_names,f_names in os.walk(path):
            for f in f_names:
                # if not re.match('^[0-9]+$', f):
                if not re.search('json$', f):
                    print(f'Skipped: {f}')
                    continue
                fullpath = os.path.join(root, f)
                self.procFile(fullpath)




    def procFile(self, SFIn):
        # output_data is a json list of dictionaries, which is updated with every json file read from the directory

        with open(SFIn, 'r', encoding='utf-8') as input_json:
            input_list = json.load(input_json)

            for i in input_list:
                if 'langOrig' in i:
                    langOrigValue = i['langOrig']
                    self.listVals.append(langOrigValue)
                    self.output_data.append(i)

    def filtByLang(self, output_data2filter, findKey, filtVal):
        # output_list = [x for x in input_list if x['langOrig'] == 'eng']
        output_list = list(filter(lambda x: x[findKey] == filtVal, output_data2filter))
        return output_list


    def dumpOutput(self, output_data2print, output_dict_file):
        json.dump(output_data2print, output_dict_file, indent=4, ensure_ascii=False)




    # output_json_string = json.dumps(output_data)
    ## json.dump(output_data, output_dict_file, indent=4, ensure_ascii=False)

# end: class clJsonDirFindFilter

In [14]:
OJsonDirFindFilter = clJsonDirFindFilter('/content/vd17-dump/json')

['grc;lat', 'heb', 'fre', 'ita', 'grc;heb', 'lat', 'fre', 'ita', 'fre', 'fre', 'lat', 'fre', 'spa', 'lat', 'heb', 'fre', 'fre', 'spa', 'fre', 'fre', 'lat', 'ita', 'eng', 'ger', 'ita', 'grc', 'lat', 'fre', 'lat', 'ita', 'spa', 'lat', 'ger', 'fre', 'fre', 'fre', 'fre', 'ita', 'ita', 'ger', 'dut', 'lat', 'eng', 'dut', 'dut', 'eng', 'eng', 'fre', 'eng', 'swe', 'fre', 'fre', 'ita', 'ita', 'lat', 'eng', 'eng', 'fre', 'fre;dut', 'fre', 'fre', 'dut', 'dut', 'rus', 'fre', 'eng', 'eng', 'eng', 'ger', 'fre', 'lat', 'lat', 'eng', 'fre', 'ota', 'grc', 'ita;fre;dut', 'fre', 'ita', 'dut', 'fre', 'fre', 'dut', 'dut', 'lat', 'dut', 'fre', 'fre', 'lat', 'lat', 'ger', 'fre', 'lat', 'ita', 'syr;grc', 'grc', 'ita', 'lat', 'lat', 'fre', 'ger', 'grc', 'lat', 'lat', 'lat', 'lat', 'fre', 'fre;ita', 'grc', 'fre', 'fre', 'fre', 'fre', 'fre', 'fre', 'fre', 'grc', 'spa', 'fre', 'heb', 'ita', 'dut', 'lat', 'eng', 'fre', 'lat', 'cze', 'lat', 'lat', 'lat', 'lat', 'fre', 'spa', 'spa', 'fre', 'fre', 'fre', 'lat', 'heb'

## 2.1 setting up the environment for download xml and converting to json

We will try to download again the original xml for 17, check if it agrees with Dump, and then try to download 18, etc.

In [ ]:
!git clone https://github.com/hbeyer/pylib

In [ ]:
%cd /content/pylib/
!pwd

/content/pylib
/content/pylib


In [ ]:
from lib import bookwheel as bw

In [ ]:
import logging
import pickle
from lib import sru
from lib import isil
from lib import xmlreader as xr
from lib import pica

In [ ]:
%cd /content
!pwd

/content
/content


In [ ]:
# testing
# from lib import bookwheel as bw
cat = bw.Catalogue
sec = cat.get_section(2589)
print(sec)


{'start': 2511, 'end': 2738, 'group': 'Libri Varii', 'dateBegin': '1634', 'year': 1634, 'writer': 'Herzog August'}


In [ ]:
!mkdir xmlbibliography
!mkdir xmlbibliographyac

In [ ]:
!mkdir jsonbibliography

## 2.2 running the download script for RecordVD17

In [ ]:
# import logging
# import pickle
# from lib import sru
# from lib import isil
# from lib import xmlreader as xr
# from lib import pica

logging.basicConfig(level=logging.ERROR)

# Festlegen der Speicherpfade und der Datensätze pro JSON-Datei
# source_folder = "{Ordner mit den PICAXML-Dateien}/"
source_folder = "/content/xmlbibliography/"
# source_folder_ac = "{Ordner mit den PICAXML-Dateien für Gesamtaufnahmen mehrbändiger Werke (Ac-Sätze)}/"
source_folder_ac = "/content/xmlbibliography/"
# target_folder = "{Ordner zum Speichern der JSON-Dateien}/"
target_folder = "/content/jsonbibliography/"
size = 1000
limit = 350000

# Laden der Ac-Sätze und Extrahieren der Gattungsbegriffe
req = sru.Request_VD17()
num = req.prepare("pica.bbg=Ac*")
print(req.url)
print(req.numFound)
req.download(source_folder_ac)

res = {}
reader = xr.DownloadReader(source_folder_ac, "record", "info:srw/schema/5/picaXML-v1.0")

for count, node in enumerate(reader):
    rec = pica.RecordVD17(node)
    gatt = [gat for gat in rec.gatt]
    if gatt == []:
        continue
    res[rec.ppn] = gatt
    if count > 100000:
        break

with open('gattungen-ac', 'wb') as file:
    pickle.dump(res, file)

# Download der PICA-XML-Daten
req = sru.Request_VD17()
num = req.prepare("pica.bbg=(Aa* or Af*)")
print(req.url)
print(req.numFound)
req.download(source_folder)

# Auslesen und Abspeichern in JSON
with open('gattungen-ac','rb') as file:
    gatt_ac = pickle.load(file)

reader = xr.DownloadReader(source_folder, "record", "info:srw/schema/5/picaXML-v1.0")

content = []
fnn = []
setn = 1
count = 0

for no, node in enumerate(reader):
    rec = pica.RecordVD17(node)
    if rec.get_rec_type() in ["Teilband", "Teilband mit eigenem Titel"]:
        try:
            rec.gatt = gatt_ac[rec.ppn_sup]
        except:
            logging.info(f"Keine Gattungsbegriffe bei PPN {rec.ppn_sup}")
    content.append(rec)
    count += 1
    if count >= size:
        recl = pica.RecordList(content)
        fn = f"vd17-{str(setn).zfill(3)}"
        recl.to_json(target_folder + fn)
        content = []
        setn += 1
        count = 0
    if no > limit:
        break
if content != []:
    recl = pica.RecordList(content)
    fn = f"vd17-{str(setn).zfill(3)}"
    recl.to_json(target_folder + fn)


## testing commands - to be removed

In [ ]:
%cd xmlbibliography/

/content/xmlbibliography


In [ ]:
!echo $PYTHONPATH

"$/env/python"


In [ ]:
!echo $PATH

"/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/content/pylib/lib"


In [ ]:
%env PATH="/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/content/pylib/lib"

env: PATH="/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/content/pylib/lib"


In [ ]:
%env PYTHONPATH="$/env/python"

env: PYTHONPATH="$/env/python"


In [ ]:
%env PYTHONPATH="$/env/python:/content/pylib/lib/:/content/pylib/lib/bookwheel.py"
# !PYTHONPATH=. ./comet/cli/score.py

env: PYTHONPATH="$/env/python:/content/pylib/lib/:/content/pylib/lib/bookwheel.py"


In [ ]:
%cd /content/
!pwd

/content
/content


In [ ]:
!wget http://sru.k10plus.de/vd17?version=2.0&operation=searchRetrieve&query=pica.bbg=%28Aa*%20or%20Af*%29&maximumRecords=500&startRecord=1&recordSchema=picaxml

In [ ]:
!mv vd17?version=2.0 vd17_500.xml

In [ ]:
%cd /content/

/content
